In [1]:
{-# LANGUAGE OverloadedStrings #-}

import Data.ByteArray.Encoding (convertToBase, Base(Base16))
import Crypto.Hash (hashWith)
import Crypto.Hash.Algorithms (MD5)
import qualified Data.ByteString as B
import qualified Data.ByteString.UTF8 as BU

input = "lpvhkcbi" :: B.ByteString

In [2]:
import Data.ByteString.Internal (c2w, w2c)
import Data.Maybe (catMaybes)

data Direction = U | D | L | R deriving (Eq, Ord, Show)

open ls = let
    zipped = zip ls [U,D,L,R]
    in map snd $ filter fst zipped

md5 :: B.ByteString -> B.ByteString
md5 = convertToBase Base16 . hashWith (undefined :: MD5)

doors input  = let 
    possible = B.unpack $ B.take 4 $ md5 input
    in open $ map (flip B.elem "bcdef") possible

next (x,y) input U | y-1 >=0 = Just ((x, y-1), B.snoc input (c2w 'U'))
next (x,y) input D | y+1 <=3 = Just ((x, y+1), B.snoc input (c2w 'D'))
next (x,y) input L | x-1 >=0 = Just ((x-1, y), B.snoc input (c2w 'L'))
next (x,y) input R | x+1 <=3 = Just ((x+1, y), B.snoc input (c2w 'R'))
next _     _     _           = Nothing


neighbours pos input = let
    possible = doors input
    in catMaybes $ map (next pos input) possible

run (p:ps) = case p of
    ((3,3), path) -> path
    _             -> let
        paths = uncurry neighbours p
        in run (ps++paths)

run (neighbours (0,0) input)

"lpvhkcbiDUDRLRRDDR"

In [3]:
run' longest []                   = longest
run' longest (p@((3,3), path):ps) = let
    pathLength = B.length path - B.length input
    longest'   = max longest pathLength
    in run' longest' ps
run' longest (p:ps)               = let
    paths = uncurry neighbours p
    in run' longest (ps++paths)

    

run' 0 (neighbours (0, 0) input)

788